In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict
from scipy.stats import gmean,hmean

In [2]:
college=pd.read_csv('data/college.csv')

In [3]:
college.head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


In [4]:
subset='UGDS SATMTMID SATVRMID'.split()
subset

['UGDS', 'SATMTMID', 'SATVRMID']

In [5]:
college.isnull()[subset].sum()

UGDS         661
SATMTMID    6339
SATVRMID    6350
dtype: int64

In [6]:
["Columns %s has nulls: %s" % (col,college[col].hasnans) for col in subset]

['Columns UGDS has nulls: True',
 'Columns SATMTMID has nulls: True',
 'Columns SATVRMID has nulls: True']

In [7]:
college2=college.dropna(how='any',subset=subset)

In [8]:
college.shape , college2.shape

((7535, 27), (1184, 27))

In [9]:
# calculating weighted average - in this case, apply() returns only a single scalar for each group

In [10]:
def weighted_math_average(df):
    weighted_math=df.UGDS*df.SATMTMID
    return int(weighted_math.sum()/df.UGDS.sum())

In [11]:
college2.groupby('STABBR').apply(weighted_math_average).head()

STABBR
AK    503
AL    536
AR    529
AZ    569
CA    564
dtype: int64

In [12]:
# calculating weighted average - in this case, apply() returns only a new DataFrame object for each group

In [31]:
def return_weighted_df(df):
    columns='SATMTMID SATVRMID'.split()
    rows='arithmetic weighted geometric harmonic'.split()
    new_df=pd.DataFrame(index=rows,columns=columns)
    for col in columns:
        arithmetic=df[col].mean()
        weighted=np.average(df[col],weights=df.UGDS)
        geometric=gmean(df[col])
        harmonic=hmean(df[col])
        new_df[col]=[arithmetic,weighted,geometric,harmonic]
    new_df['count']=len(df)
    return new_df.astype(int)

In [34]:
college2.groupby('STABBR').apply(return_weighted_df).head(12)

SATMTMID  SATVRMID  count
STABBR                                      
AK     arithmetic       503       555      1
       weighted         503       555      1
       geometric        503       555      1
       harmonic         503       555      1
AL     arithmetic       504       508     21
       weighted         536       533     21
       geometric        500       505     21
       harmonic         497       502     21
AR     arithmetic       515       491     16
       weighted         529       504     16
       geometric        514       489     16
       harmonic         513       487     16